In [1]:
import numpy as np
import plotly.graph_objects as go

rng = np.random.default_rng(42)
t = np.arange(60)
y = np.sin(t / 6) + 0.2 * rng.normal(size=t.size)

fh = 12
cutoff = len(t) - fh

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=t[:cutoff], y=y[:cutoff], mode="lines", name="history")
)
fig.add_trace(
    go.Scatter(
        x=t[cutoff:],
        y=y[cutoff:],
        mode="lines",
        name="forecast horizon",
        line=dict(dash="dash"),
    )
)
fig.add_shape(
    type="line",
    x0=cutoff - 0.5,
    x1=cutoff - 0.5,
    y0=min(y) - 0.5,
    y1=max(y) + 0.5,
    line=dict(color="gray", dash="dot"),
)
fig.update_layout(
    title="Forecast horizon relative to the cutoff",
    xaxis_title="time",
    yaxis_title="value",
)
fig.show()


In [2]:
import pandas as pd
import plotly.express as px


def _resolve_all_estimators():
    try:
        from sktime.registry import all_estimators
        return all_estimators
    except Exception:
        from sktime.utils.discovery import all_estimators
        return all_estimators


def _safe_tags(cls):
    if hasattr(cls, "get_class_tags"):
        try:
            return cls.get_class_tags()
        except Exception:
            return {}
    return {}


def _matches_scitype(value, target):
    if value is None:
        return False
    if isinstance(value, (list, tuple, set)):
        return target in value
    return value == target


def _top_tag_keys(tag_dicts, limit=8):
    counts = {}
    for tags in tag_dicts:
        for key, value in tags.items():
            if key in {"scitype", "estimator_type", "task", "learning_type"}:
                continue
            if isinstance(value, (bool, int, float, str)):
                counts[key] = counts.get(key, 0) + 1
    return [
        key
        for key, _ in sorted(counts.items(), key=lambda kv: (-kv[1], kv[0]))[:limit]
    ]


try:
    import sktime  # noqa: F401

    all_estimators = _resolve_all_estimators()
    SKTIME_AVAILABLE = True
except Exception as exc:
    SKTIME_AVAILABLE = False
    _err = exc

if not SKTIME_AVAILABLE:
    print("sktime not installed. Install with `pip install sktime` to populate the catalog.")
    print("Error:", _err)


sktime not installed. Install with `pip install sktime` to populate the catalog.
Error: No module named 'sktime'


In [3]:
if SKTIME_AVAILABLE:
    target = "forecaster"
    estimators = all_estimators()
    records = []
    tag_dicts = []
    for name, cls in estimators:
        tags = _safe_tags(cls)
        scitype = tags.get("scitype") or tags.get("estimator_type")
        if not _matches_scitype(scitype, target):
            continue
        records.append(
            {
                "name": name,
                "class": cls.__name__,
                "module": cls.__module__,
                "module_family": ".".join(cls.__module__.split(".")[:3]),
            }
        )
        tag_dicts.append(tags)

    tag_keys = _top_tag_keys(tag_dicts, limit=8)
    rows = []
    for record, tags in zip(records, tag_dicts):
        row = record.copy()
        for key in tag_keys:
            row[key] = tags.get(key)
        rows.append(row)

    df = pd.DataFrame(rows).sort_values("name")
    df.head(20)


In [4]:
if SKTIME_AVAILABLE and not df.empty:
    counts = (
        df["module_family"]
        .value_counts()
        .reset_index()
        .rename(columns={"index": "module_family", "module_family": "count"})
    )
    fig = px.bar(
        counts,
        x="module_family",
        y="count",
        title="Forecasters by module family",
    )
    fig.show()

    fig = px.treemap(
        df,
        path=["module_family", "name"],
        title="Forecasters catalog (module family -> estimator)",
    )
    fig.show()

    bool_tags = []
    for key in df.columns:
        if key in {"name", "class", "module", "module_family"}:
            continue
        series = df[key].dropna()
        if not series.empty and series.isin([True, False]).all():
            bool_tags.append(key)

    if bool_tags:
        summary = pd.DataFrame(
            {
                "tag": bool_tags,
                "share_true": [df[tag].mean() for tag in bool_tags],
            }
        )
        fig = px.bar(
            summary,
            x="tag",
            y="share_true",
            title="Share of estimators with tag=True",
        )
        fig.show()
